## In this DOC, the linear retrun rate is used.

In [5]:
# -*- coding: utf-8 -*-
"""
@Created at 2022/7/2 15:38
@Author: Kurt
@file:uniform.py
@Desc:
"""
from collections import Counter
import numpy as np
import logging

logging.basicConfig()
logger = logging.getLogger("uniform")
logger.setLevel(logging.DEBUG)

EPSILON = 0.000001


def myround(num):
    num = num if abs(num) > EPSILON else 0
    return num


def get_return_probability(m, p):
    return m * p


def utility_tie_online(loc, c, con, m, p):
    """
    In this function, the tie is broken by assuming consumers buy online directly
    :param loc: value of theta
    """
    u_o = 1 / 2 * (loc - p) - 1 / 2 * (1 - get_return_probability(m, p)) * p - con
    u_s = 1 / 2 * (loc - p) - c
    if myround(u_o - u_s) >= 0:
        if myround(u_o) >= 0:
            return "o"
        else:
            return "l"
    else:
        if myround(u_s) >= 0:
            return "s"
        else:
            return "l"


def utility_tie_offline(loc, c, con, m, p):
    """
    In this function, the tie is broken by assuming consumers visit the store
    :param loc: value of theta
    """
    u_o = 1 / 2 * (loc - p) - 1 / 2 * (1 - get_return_probability(m, p)) * p - con
    u_s = 1 / 2 * (loc - p) - c
    if myround(u_o - u_s) > 0:
        if myround(u_o) >= 0:
            return "o"
        else:
            return "l"
    else:
        if myround(u_s) >= 0:
            return "s"
        else:
            return "l"


def get_demand(behaviors):
    total = len(behaviors)
    count = Counter(behaviors)
    alpha_o = count['o'] / total
    alpha_s = count['s'] / total
    alpha_l = count['l'] / total
    assert myround(alpha_o + alpha_s + alpha_l - 1) == 0

    return alpha_o, alpha_s


def simulate_behavior(consumers, c, con, m, p):
    # if consumers are indifferent between buying online directly and visiting the store,
    # we break the tie by maximizing the retailer's profit
    if myround(c - con - 1 / 2 * (1 - get_return_probability(m, p)) * p) == 0:
        behaviors_tie_online = [utility_tie_online(loc=consumer, c=c, con=con, m=m, p=p) for consumer in consumers]
        behaviors_tie_offline = [utility_tie_offline(loc=consumer, c=c, con=con, m=m, p=p) for consumer in consumers]

        return behaviors_tie_online, behaviors_tie_offline
    else:
        # if there is no tie, utility_tie_online and utility_tie_offline are equivalent.
        behaviors = [utility_tie_online(loc=consumer, c=c, con=con, m=m, p=p) for consumer in consumers]

        return behaviors


def cal_profit(m, p, cr, behaviors):
    alpha_o, alpha_s = get_demand(behaviors)
    online_profit = alpha_o * (1 / 2 * p + 1 / 2 * (
                (1 - get_return_probability(m=m, p=p)) * p - get_return_probability(m=m, p=p) * cr))  # w.p. 1/2, b=b_H.
    store_profit = alpha_s * 1 / 2 * p  # w.p. 1/2, b=b_H
    profit = 1 / 2 * store_profit + 1 / 2 * online_profit  # w.p. 1/2, a=a_H
    return profit

In [6]:
class uniform:

    def __init__(self, c, con, cr, return_prop, step=0.001, density=0.001):
        """
        :param c: offline shopping cost
        :param con: online shopping cost
        :param cr: retailer return cost
        :param m: return probability of direct online purchase
        :param step: step size of price
        :param density=0.001: density of consumers
        """
        self.p = 0
        self.profit = 0
        self.solve(c=c, con=con, cr=cr, return_prop=return_prop, step=step, density=density)

    def solve(self, c, con, cr, return_prop, step, density):
        consumers = np.arange(0, 1, density)

        optimal_profit = 0
        optimal_price = 0
        for p in np.arange(0.0001, 1, step):
            if isinstance(return_prop, str):
                m = 1 / (2 * p)
            else:
                m = return_prop
            logger.debug("current loop: p={:.3f}".format(p))
#             if myround(1 / 2 * m * p * p - 1 / 2 * p + c - con) == 0:
            if myround(c - con - 1/2*(1-get_return_probability(m, p))*p)==0:
                behaviors_tie_online, behaviors_tie_offline = simulate_behavior(consumers=consumers,
                                                                                c=c, con=con, m=m, p=p)
                profit_tie_online = cal_profit(m=m, p=p, cr=cr, behaviors=behaviors_tie_online)
                profit_tie_offline = cal_profit(m=m, p=p, cr=cr, behaviors=behaviors_tie_offline)
                profit = max(profit_tie_online, profit_tie_offline)
            else:
                behaviors = simulate_behavior(consumers=consumers, c=c, con=con, m=m, p=p)
                profit = cal_profit(m=m, p=p, cr=cr, behaviors=behaviors)

            if profit - optimal_profit > 0:
                optimal_profit = profit
                optimal_price = p
        self.p = optimal_price
        self.profit = optimal_profit

In [9]:
if __name__ == "__main__":
    c = 0.10
    cr = 0.32
    con = 0.05
    m = 1 / 4
    uniform_ins = uniform(c=c, con=con, cr=cr, m=m, step=0.001, density=0.001)
    print(uniform_ins.p, uniform_ins.profit)


DEBUG:uniform:current loop: p=0.000
DEBUG:uniform:current loop: p=0.001
DEBUG:uniform:current loop: p=0.002
DEBUG:uniform:current loop: p=0.003
DEBUG:uniform:current loop: p=0.004
DEBUG:uniform:current loop: p=0.005
DEBUG:uniform:current loop: p=0.006
DEBUG:uniform:current loop: p=0.007
DEBUG:uniform:current loop: p=0.008
DEBUG:uniform:current loop: p=0.009
DEBUG:uniform:current loop: p=0.010
DEBUG:uniform:current loop: p=0.011
DEBUG:uniform:current loop: p=0.012
DEBUG:uniform:current loop: p=0.013
DEBUG:uniform:current loop: p=0.014
DEBUG:uniform:current loop: p=0.015
DEBUG:uniform:current loop: p=0.016
DEBUG:uniform:current loop: p=0.017
DEBUG:uniform:current loop: p=0.018
DEBUG:uniform:current loop: p=0.019
DEBUG:uniform:current loop: p=0.020
DEBUG:uniform:current loop: p=0.021
DEBUG:uniform:current loop: p=0.022
DEBUG:uniform:current loop: p=0.023
DEBUG:uniform:current loop: p=0.024
DEBUG:uniform:current loop: p=0.025
DEBUG:uniform:current loop: p=0.026
DEBUG:uniform:current loop: 

DEBUG:uniform:current loop: p=0.228
DEBUG:uniform:current loop: p=0.229
DEBUG:uniform:current loop: p=0.230
DEBUG:uniform:current loop: p=0.231
DEBUG:uniform:current loop: p=0.232
DEBUG:uniform:current loop: p=0.233
DEBUG:uniform:current loop: p=0.234
DEBUG:uniform:current loop: p=0.235
DEBUG:uniform:current loop: p=0.236
DEBUG:uniform:current loop: p=0.237
DEBUG:uniform:current loop: p=0.238
DEBUG:uniform:current loop: p=0.239
DEBUG:uniform:current loop: p=0.240
DEBUG:uniform:current loop: p=0.241
DEBUG:uniform:current loop: p=0.242
DEBUG:uniform:current loop: p=0.243
DEBUG:uniform:current loop: p=0.244
DEBUG:uniform:current loop: p=0.245
DEBUG:uniform:current loop: p=0.246
DEBUG:uniform:current loop: p=0.247
DEBUG:uniform:current loop: p=0.248
DEBUG:uniform:current loop: p=0.249
DEBUG:uniform:current loop: p=0.250
DEBUG:uniform:current loop: p=0.251
DEBUG:uniform:current loop: p=0.252
DEBUG:uniform:current loop: p=0.253
DEBUG:uniform:current loop: p=0.254
DEBUG:uniform:current loop: 

DEBUG:uniform:current loop: p=0.456
DEBUG:uniform:current loop: p=0.457
DEBUG:uniform:current loop: p=0.458
DEBUG:uniform:current loop: p=0.459
DEBUG:uniform:current loop: p=0.460
DEBUG:uniform:current loop: p=0.461
DEBUG:uniform:current loop: p=0.462
DEBUG:uniform:current loop: p=0.463
DEBUG:uniform:current loop: p=0.464
DEBUG:uniform:current loop: p=0.465
DEBUG:uniform:current loop: p=0.466
DEBUG:uniform:current loop: p=0.467
DEBUG:uniform:current loop: p=0.468
DEBUG:uniform:current loop: p=0.469
DEBUG:uniform:current loop: p=0.470
DEBUG:uniform:current loop: p=0.471
DEBUG:uniform:current loop: p=0.472
DEBUG:uniform:current loop: p=0.473
DEBUG:uniform:current loop: p=0.474
DEBUG:uniform:current loop: p=0.475
DEBUG:uniform:current loop: p=0.476
DEBUG:uniform:current loop: p=0.477
DEBUG:uniform:current loop: p=0.478
DEBUG:uniform:current loop: p=0.479
DEBUG:uniform:current loop: p=0.480
DEBUG:uniform:current loop: p=0.481
DEBUG:uniform:current loop: p=0.482
DEBUG:uniform:current loop: 

DEBUG:uniform:current loop: p=0.684
DEBUG:uniform:current loop: p=0.685
DEBUG:uniform:current loop: p=0.686
DEBUG:uniform:current loop: p=0.687
DEBUG:uniform:current loop: p=0.688
DEBUG:uniform:current loop: p=0.689
DEBUG:uniform:current loop: p=0.690
DEBUG:uniform:current loop: p=0.691
DEBUG:uniform:current loop: p=0.692
DEBUG:uniform:current loop: p=0.693
DEBUG:uniform:current loop: p=0.694
DEBUG:uniform:current loop: p=0.695
DEBUG:uniform:current loop: p=0.696
DEBUG:uniform:current loop: p=0.697
DEBUG:uniform:current loop: p=0.698
DEBUG:uniform:current loop: p=0.699
DEBUG:uniform:current loop: p=0.700
DEBUG:uniform:current loop: p=0.701
DEBUG:uniform:current loop: p=0.702
DEBUG:uniform:current loop: p=0.703
DEBUG:uniform:current loop: p=0.704
DEBUG:uniform:current loop: p=0.705
DEBUG:uniform:current loop: p=0.706
DEBUG:uniform:current loop: p=0.707
DEBUG:uniform:current loop: p=0.708
DEBUG:uniform:current loop: p=0.709
DEBUG:uniform:current loop: p=0.710
DEBUG:uniform:current loop: 

DEBUG:uniform:current loop: p=0.912
DEBUG:uniform:current loop: p=0.913
DEBUG:uniform:current loop: p=0.914
DEBUG:uniform:current loop: p=0.915
DEBUG:uniform:current loop: p=0.916
DEBUG:uniform:current loop: p=0.917
DEBUG:uniform:current loop: p=0.918
DEBUG:uniform:current loop: p=0.919
DEBUG:uniform:current loop: p=0.920
DEBUG:uniform:current loop: p=0.921
DEBUG:uniform:current loop: p=0.922
DEBUG:uniform:current loop: p=0.923
DEBUG:uniform:current loop: p=0.924
DEBUG:uniform:current loop: p=0.925
DEBUG:uniform:current loop: p=0.926
DEBUG:uniform:current loop: p=0.927
DEBUG:uniform:current loop: p=0.928
DEBUG:uniform:current loop: p=0.929
DEBUG:uniform:current loop: p=0.930
DEBUG:uniform:current loop: p=0.931
DEBUG:uniform:current loop: p=0.932
DEBUG:uniform:current loop: p=0.933
DEBUG:uniform:current loop: p=0.934
DEBUG:uniform:current loop: p=0.935
DEBUG:uniform:current loop: p=0.936
DEBUG:uniform:current loop: p=0.937
DEBUG:uniform:current loop: p=0.938
DEBUG:uniform:current loop: 

0.4 0.04000000000000001
